# <center> Groceries Market Basket and RFM Analysis

In [1]:
import os
import plotly.express as px
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import seaborn as sns
import operator as op
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules



# Dataset Exploration

In [2]:
# Loading dataset
data = pd.read_csv('Groceries_dataset.csv')
data.head()

,Member_number,Date,itemDescription
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


**As you can see from the first 5 rows of the dataset, we have 3 different columns: Member_number is a number that is unique for each customer. Date represents the date of the transaction, and finally itemDescription represents the corresponding product bought for this date.**

In [4]:
# To check the data types of the columns, I use info() function. According to result below, Member_number and Date columns are not in correct datatype.
#For the following steps, I will change them into correct form.
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Member_number    38765 non-null  int64 
 1   Date             38765 non-null  object
 2   itemDescription  38765 non-null  object
dtypes: int64(1), object(2)
memory usage: 908.7+ KB


In [5]:
# Renaming the columns, because it would be easier to understand what that means.
data.columns = ['memberID', 'Date', 'itemName']
data.head()

,memberID,Date,itemName
0,1808,21-07-2015,tropical fruit
1,2552,05-01-2015,whole milk
2,2300,19-09-2015,pip fruit
3,1187,12-12-2015,other vegetables
4,3037,01-02-2015,whole milk


In [6]:
# Checking for the missing values
nan_values = data.isna().sum()
nan_values

memberID    0
Date        0
itemName    0
dtype: int64

**According to result above, there is no missing values for all columns.**

In [7]:
# Converting Date column into correct datatype which is datetime
data.Date = pd.to_datetime(data.Date)
data.memberID = data['memberID'].astype('str')
data.info()  # They are in correct datatype now

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38765 entries, 0 to 38764
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   memberID  38765 non-null  object        
 1   Date      38765 non-null  datetime64[ns]
 2   itemName  38765 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 908.7+ KB


/tmp/ipykernel_12716/3007076359.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data.Date = pd.to_datetime(data.Date)


# Data Visualization

# Number of Sales Weekly


In [8]:
Sales_weekly = data.resample('w', on='Date').size()
fig = px.line(data, x=Sales_weekly.index, y=Sales_weekly,
              labels={'y': 'Number of Sales',
                     'x': 'Date'})
fig.update_layout(title_text='Number of Sales Weekly',
                  title_x=0.5, title_font=dict(size=18)) 
fig.show()

/tmp/ipykernel_12716/1638432699.py:1: FutureWarning: 'w' is deprecated and will be removed in a future version, please use 'W' instead.
  Sales_weekly = data.resample('w', on='Date').size()


# Number of Customers Weekly

In [10]:
Unique_customer_weekly = data.resample('w', on='Date').memberID.nunique()
fig = px.line(Unique_customer_weekly, x=Unique_customer_weekly.index, y=Unique_customer_weekly,
              labels={'y': 'Number of Customers'})
fig.update_layout(title_text='Number of Customers Weekly',
                  title_x=0.5, title_font=dict(size=18))
fig.show()

/tmp/ipykernel_12716/2131395614.py:1: FutureWarning:

'w' is deprecated and will be removed in a future version, please use 'W' instead.



# Sales per Customer Weekly

In [11]:
Sales_per_Customer = Sales_weekly / Unique_customer_weekly
fig = px.line(Sales_per_Customer, x=Sales_per_Customer.index, y=Sales_per_Customer,
              labels={'y': 'Sales per Customer Ratio'})
fig.update_layout(title_text='Sales per Customer Weekly',
                  title_x=0.5, title_font=dict(size=18))
fig.update_yaxes(rangemode="tozero")
fig.show()

**Sales per Customer ratio is almost constant until the end of December 2014 with the number of 2.2. Fortunately, after this week this parameter increases to approximately 3 sales per customer value. And it keeps going until the end of the date range. It has a maximum of 3.3 sales per customer in at first week of March 2015. The reason for that might be the constant number of sales weekly but decreasing number of customers for these date range. You can easily see the decrease in the number of customers weekly after March 2015 from the "Number of Customers Weekly" graph.**

# Frequency of the Items Sold

In [14]:
Frequency_of_items = data.groupby("itemName").size().reset_index(name='count')
fig = px.treemap(Frequency_of_items, path=['itemName'], values='count')
fig.update_layout(title_text='Frequency of the Items Sold',
                  title_x=0.5, title_font=dict(size=18)
                  )
fig.update_traces(textinfo="label+value")
fig.show()

**Whole milk, vegetables and rolls/buns are top 3 products that sold for this groceries.**

# Top Customers regarding Number of Items bought

In [13]:
user_item = data.groupby("memberID").size().reset_index(name='count') \
    .sort_values(by='count', ascending=False)
fig = px.bar(user_item.head(25), x='memberID', y='count',
             labels={'y': 'Number of Sales',
                     'count': 'Number of Items Bought'},
             color='count')
fig.update_layout(title_text='Top 20 Customers regarding Number of Items Bought',
                  title_x=0.5, title_font=dict(size=18))
fig.update_traces(marker=dict(line=dict(color='#000000', width=1)))
fig.show()

**Customer 3180 looks like our the best customer :)**

# Number of Sales per Discrete Week Days


In [20]:
day = data.groupby(data['Date'].dt.strftime('%A'))['itemName'].count()
fig = px.bar(day, x=day.index, y=day, color=day,
             labels={'y': 'Number of Sales',
                     'Date': 'Week Days'})
fig.update_layout(title_text='Number of Sales per Discrete Week Days',
                  title_x=0.5, title_font=dict(size=18))
fig.update_traces(marker=dict(line=dict(color='#000000', width=1)))
fig.show()

**According to the graph above, there are more sales on Thursdays. Sunday and Wednesday are the following days for the most sales made.**

# Number of Sales per Discrete Months


In [21]:
month = data.groupby(data['Date'].dt.strftime('%m'))['itemName'].count()
fig = px.bar(month, x=month.index, y=month, color=month,
             labels={'y': 'Number of Sales',
                     'Date': 'Months'})
fig.update_layout(title_text='Number of Sales per Discrete Months',
                  title_x=0.5, title_font=dict(size=18))
fig.update_traces(marker=dict(line=dict(color='#000000', width=1)))
fig.show()

# Number of Sales per Discrete Month Days


In [22]:
month_day = data.groupby(data['Date'].dt.strftime('%d'))['itemName'].count()
fig = px.bar(month_day, x=month_day.index, y=month_day, color=month_day,
             labels={'y': 'Number of Sales',
                     'Date': 'Month Days'})
fig.update_layout(title_text='Number of Sales per Discrete Month Days',
                  title_x=0.5, title_font=dict(size=18))
fig.update_traces(marker=dict(line=dict(color='#000000', width=1)))
fig.show()

**Approximately each Month Day has equal Number of Sales.**

# <center> Market Basket Analysis

**Market basket analysis is a data mining technique used by retailers to increase sales by better understanding customer purchasing patterns. It involves analyzing large data sets, such as purchase history, to reveal product groupings, as well as products that are likely to be purchased together [1].**

# Let's Create Baskets

In [23]:
baskets = data.groupby(['memberID', 'itemName'])['itemName'].count().unstack().fillna(0).reset_index()
baskets.head()

itemName,memberID,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,1000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0
1,1001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0
2,1002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,1003,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0


**According to the query above, we created the customers-items matrix. Each row represents the transactions of each customer, and each column represents the items bought. The numbers corresponding to the matrix represent the number of times that item is bought by the individual user.**

In [24]:
# Let's check the most sold -item which is whole milk- has the same number of sales as we discussed above in the treemap.
baskets['whole milk'].sum()

# Yep it satisfies our results.

2502.0

In [25]:
# Encoding the items that sold more than 1
def one_hot_encoder(k):
    if k <= 0:
        return 0
    if k >= 1:
        return 1

In [26]:
baskets_final = baskets.iloc[:, 1:baskets.shape[1]].applymap(one_hot_encoder)
baskets_final.head()

/tmp/ipykernel_12716/941720042.py:1: FutureWarning:

DataFrame.applymap has been deprecated. Use DataFrame.map instead.



itemName,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,1,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,1,0,1,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


**Final form of our customer-item matrix.**

In [27]:
# Finding the most frequent items sold together
frequent_itemsets = apriori(baskets_final, min_support=0.025, use_colnames=True, max_len=3).sort_values(by='support')
frequent_itemsets.head(25)

/home/tony/anaconda3/lib/python3.12/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning:

DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type



,support,itemsets
161,0.025141,"(shopping bags, butter)"
69,0.025141,(spread cheese)
405,0.025141,"(whole milk, sliced cheese)"
412,0.025141,"(whole milk, specialty bar)"
85,0.025141,"(beef, pip fruit)"
248,0.025141,"(shopping bags, domestic eggs)"
467,0.025141,"(whole milk, tropical fruit, citrus fruit)"
480,0.025141,"(whole milk, yogurt, frankfurter)"
119,0.025141,"(chocolate, bottled water)"
239,0.025141,"(dessert, root vegetables)"


**As you can see from the results above, the most items that appeared together are butter and shopping bags, and spread cheese, and so on.**

In [28]:
# Creating association rules for indicating astecedent and consequent items
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1).sort_values('lift', ascending=False)
rules = rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']]
rules.head(25)

,antecedents,consequents,support,confidence,lift
878,(sausage),"(yogurt, rolls/buns)",0.035659,0.173101,1.554717
875,"(yogurt, rolls/buns)",(sausage),0.035659,0.320276,1.554717
455,"(whole milk, root vegetables)",(shopping bags),0.029246,0.258503,1.536046
458,(shopping bags),"(whole milk, root vegetables)",0.029246,0.173780,1.536046
876,"(sausage, rolls/buns)",(yogurt),0.035659,0.433022,1.530298
877,(yogurt),"(sausage, rolls/buns)",0.035659,0.126020,1.530298
943,"(other vegetables, yogurt)",(sausage),0.037199,0.309168,1.500795
946,(sausage),"(other vegetables, yogurt)",0.037199,0.180573,1.500795
632,(shopping bags),"(other vegetables, soda)",0.031042,0.184451,1.485518
629,"(other vegetables, soda)",(shopping bags),0.031042,0.250000,1.485518


**As you can see from the result above:**

**sausage --> yogurt, rolls/buns**

**root vegetables, whole milk --> shopping bags**

**rolls/buns, soda --> sausage**

**butter, whole milk --> yogurt,**

**and etc. have strong relationships.**

# <center> RFM Analysis

# Recency

In [29]:
# Lets Start with the calculate the Recency

# Finding last purchase date of each customer
Recency = data.groupby(by='memberID')['Date'].max().reset_index()
Recency.columns = ['memberID', 'LastDate']
Recency.head()

,memberID,LastDate
0,1000,2015-11-25
1,1001,2015-05-02
2,1002,2015-08-30
3,1003,2015-02-10
4,1004,2015-12-02


In [30]:
# Finding last date for our dataset
last_date_dataset = Recency['LastDate'].max()
last_date_dataset

Timestamp('2015-12-30 00:00:00')

In [31]:
# Calculating Recency by subtracting (last transaction date of dataset) and (last purchase date of each customer)
Recency['Recency'] = Recency['LastDate'].apply(lambda x: (last_date_dataset - x).days)
Recency.head()

,memberID,LastDate,Recency
0,1000,2015-11-25,35
1,1001,2015-05-02,242
2,1002,2015-08-30,122
3,1003,2015-02-10,323
4,1004,2015-12-02,28


# Recency Distribution of the Customers

In [32]:
fig = px.histogram(Recency, x='Recency', opacity=0.85, marginal='box')
fig.update_traces(marker=dict(line=dict(color='#000000', width=1)))
fig.update_layout(title_text='Recency Distribution of the Customers',
                  title_x=0.5, title_font=dict(size=20))
fig.show()

**According to Recency Historgram of the Customers, we can see that most of the customers are distributed between 57th-280th day of thier last purchase.**

# Visit Frequency

In [33]:
# Frequency of the customer visits
Frequency = data.drop_duplicates(['Date', 'memberID']).groupby(by=['memberID'])['Date'].count().reset_index()
Frequency.columns = ['memberID', 'Visit_Frequency']
Frequency.head()

,memberID,Visit_Frequency
0,1000,5
1,1001,5
2,1002,4
3,1003,4
4,1004,8


# Visit Frequency Distribution of the Customers


In [34]:
fig = px.histogram(Frequency, x='Visit_Frequency', opacity=0.85, marginal='box')
fig.update_traces(marker=dict(line=dict(color='#000000', width=1)))
fig.update_layout(title_text='Visit Frequency Distribution of the Customers',
                  title_x=0.5, title_font=dict(size=20))
fig.show()

**According to Frequency Historgram of the Customers, we can see that most of the customers are distributed between their 2nd-5th visit to the Groceries Store.**

# Monetary

**Due to our dataset, we have no data regarding the price of the products. Therefore I will consider the number of item bought per user as the Monetary value.**

In [35]:
Monetary = data.groupby(by="memberID")['itemName'].count().reset_index()
Monetary.columns = ['memberID', 'Monetary']
Monetary.head()

,memberID,Monetary
0,1000,13
1,1001,12
2,1002,8
3,1003,8
4,1004,21


In [36]:
# I assumed each item has equal price and price is 10
Monetary['Monetary'] = Monetary['Monetary'] * 10
Monetary.head()

,memberID,Monetary
0,1000,130
1,1001,120
2,1002,80
3,1003,80
4,1004,210


# Monetary Distribution of the Customers

In [37]:
fig = px.histogram(Monetary, x='Monetary', opacity=0.85, marginal='box',
                   labels={'itemName': 'Monetary'})
fig.update_traces(marker=dict(line=dict(color='#000000', width=1)))
fig.update_layout(title_text='Monetary Distribution of the Customers',
                  title_x=0.5, title_font=dict(size=20))
fig.show()

In [38]:
# Combining all scores into one DataFrame
RFM = pd.concat([Recency['memberID'], Recency['Recency'], Frequency['Visit_Frequency'], Monetary['Monetary']], axis=1)
RFM.head()

,memberID,Recency,Visit_Frequency,Monetary
0,1000,35,5,130
1,1001,242,5,120
2,1002,122,4,80
3,1003,323,4,80
4,1004,28,8,210


# RFM Scores

In [39]:
# 5-5 score = the best customers
RFM['Recency_quartile'] = pd.qcut(RFM['Recency'], 5, [5, 4, 3, 2, 1])
RFM['Frequency_quartile'] = pd.qcut(RFM['Visit_Frequency'], 5, [1, 2, 3, 4, 5])

RFM['RF_Score'] = RFM['Recency_quartile'].astype(str) + RFM['Frequency_quartile'].astype(str)
RFM.head()

,memberID,Recency,Visit_Frequency,Monetary,Recency_quartile,Frequency_quartile,RF_Score
0,1000,35,5,130,5,4,54
1,1001,242,5,120,2,4,24
2,1002,122,4,80,3,3,33
3,1003,323,4,80,2,3,23
4,1004,28,8,210,5,5,55


In [41]:

segt_map = {  # Segmentation Map [Ref]
    r'[1-2][1-2]': 'hibernating',
    r'[1-2][3-4]': 'at risk',
    r'[1-2]5': 'can\'t loose',
    r'3[1-2]': 'about to sleep',
    r'33': 'need attention',
    r'[3-4][4-5]': 'loyal customers',
    r'41': 'promising',
    r'51': 'new customers',
    r'[4-5][2-3]': 'potential loyalists',
    r'5[4-5]': 'champions'
}

RFM['RF_Segment'] = RFM['RF_Score'].replace(segt_map, regex=True)
RFM.head()

,memberID,Recency,Visit_Frequency,Monetary,Recency_quartile,Frequency_quartile,RF_Score,RF_Segment
0,1000,35,5,130,5,4,54,champions
1,1001,242,5,120,2,4,24,at risk
2,1002,122,4,80,3,3,33,need attention
3,1003,323,4,80,2,3,23,at risk
4,1004,28,8,210,5,5,55,champions


# Distribution of the RFM Segments


In [42]:
x = RFM.RF_Segment.value_counts()
fig = px.treemap(x, path=[x.index], values=x)
fig.update_layout(title_text='Distribution of the RFM Segments', title_x=0.5,
                  title_font=dict(size=20))
fig.update_traces(textinfo="label+value+percent root")
fig.show()

**According to our RFM analysis, most of the customers are segmented into hibernating group which is they are not visiting our store often and it passed pretty much time after their last visit. You can find detailed information about these segments in the references part of this notebook.**

# Relationship between Visit_Frequency and Recency


In [43]:
fig = px.scatter(RFM, x="Visit_Frequency", y="Recency", color='RF_Segment',
                 labels={"math score": "Math Score",
                         "writing score": "Writing Score"})
fig.update_layout(title_text='Relationship between Visit_Frequency and Recency',
                  title_x=0.5, title_font=dict(size=20))
fig.show()

**As you can see the graph above, when the visit frequency is low and the recency is high, customers are most likely to segmented into hibernating segment. In contrast, when they are visiting our store frequently, and their recency is low, they are most likely to segmented into champions segment which is the best segment for all of the customer segments.**